<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#load-data" data-toc-modified-id="load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>load data</a></span><ul class="toc-item"><li><span><a href="#read-in-the-files" data-toc-modified-id="read-in-the-files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>read in the files</a></span><ul class="toc-item"><li><span><a href="#data-from-linear-and-relu-activations" data-toc-modified-id="data-from-linear-and-relu-activations-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>data from linear and relu activations</a></span></li></ul></li><li><span><a href="#create-pandas-table" data-toc-modified-id="create-pandas-table-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>create pandas table</a></span></li></ul></li><li><span><a href="#filter-out-bad-training-losses" data-toc-modified-id="filter-out-bad-training-losses-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>filter out bad training losses</a></span></li><li><span><a href="#determine-the-lambda-parameter-that-gets-the-best-validation-MSE-for-each-(r,n,L)" data-toc-modified-id="determine-the-lambda-parameter-that-gets-the-best-test-MSE-for-each-(r,n,L)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>determine the lambda parameter that gets the best test MSE for each (r,n,L)</a></span></li><li><span><a href="#Generalization-MSE" data-toc-modified-id="Generalization-MSE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generalization MSE</a></span><ul class="toc-item"><li><span><a href="#generate-data" data-toc-modified-id="generate-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>generate data</a></span></li><li><span><a href="#compute-MSE" data-toc-modified-id="compute-MSE-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>compute MSE</a></span></li></ul></li><li><span><a href="#Out-of-Distribution-MSE" data-toc-modified-id="Out-of-Distribution-MSE-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Out of Distribution MSE</a></span><ul class="toc-item"><li><span><a href="#generate-data" data-toc-modified-id="generate-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>generate data</a></span></li><li><span><a href="#compute-MSE" data-toc-modified-id="compute-MSE-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>compute MSE</a></span></li></ul></li><li><span><a href="#Active-Subspace" data-toc-modified-id="Active-Subspace-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Active Subspace</a></span><ul class="toc-item"><li><span><a href="#evaluate-gradients-and-compute-singular-values-and-active-subspaces" data-toc-modified-id="evaluate-gradients-and-compute-singular-values-and-active-subspaces-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>evaluate gradients and compute singular values and active subspaces</a></span></li><li><span><a href="#plot-of-singular-values" data-toc-modified-id="plot-of-singular-values-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>plot of singular values</a></span></li></ul></li><li><span><a href="#determine-the-L-parameter-that-gets-the-best-test-MSE-for-each-(r,n)" data-toc-modified-id="determine-the-L-parameter-that-gets-the-best-test-MSE-for-each-(r,n)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>determine the L parameter that gets the best test MSE for each (r,n)</a></span></li><li><span><a href="#Plots-of-L-vs-Test-error-and-n-vs-Generalization-metrics-with/without-linear-layers" data-toc-modified-id="Plots-of-L-vs-Validation-error-and-n-vs-Generalization-metrics-with/without-linear-layers-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Plots of L vs Test error and n vs Generalization metrics with/without linear layers</a></span></li><li><span><a href="#Final-Table" data-toc-modified-id="Final-Table-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Final Table</a></span></li><li><span><a href="#Training-Time-Plots" data-toc-modified-id="Training-Time-Plots-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Training Time Plots</a></span><ul class="toc-item"><li><span><a href="#Train-MSE-v-Epoch" data-toc-modified-id="Train-MSE-v-Epoch-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Train MSE v Epoch</a></span></li><li><span><a href="#Weight-Decay-v-Epoch" data-toc-modified-id="Weight-Decay-v-Epoch-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Weight Decay v Epoch</a></span></li><li><span><a href="#learning-rates" data-toc-modified-id="learning-rates-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>learning rates</a></span></li></ul></li></ul></div>

# imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import ortho_group
from scipy.stats import linregress
from scipy import linalg as la
from torch import nn
import torch
import os
from matplotlib.lines import Line2D
from scipy.stats import sem
from mpl_toolkits import mplot3d

# load data

## read in the files

In [2]:
rnvals = [(1,64),(1,128),(1,256),(1,512),(1,1024),(1,2048),
          (2,64),(2,128),(2,256),(2,512),(2,1024),(2,2048)]
Ls = [2,3,4,5,6,7,8,9]
rs = [1,2]
wds = [1e-3,1e-4,1e-5]
labelnoise = [0,0.25,0.5,1]
epochs = 60100
job_name = "new_targets"

In [3]:
testMSEs = {}
trainMSEs = {}
weightdecays = {}
learningrates = {}
files_found_list = []
for r,n in rnvals:
    for L in Ls:
        for wd in wds:
            for ln in labelnoise:
                paramname = job_name+f"_labelnoise{ln}"+f"/N{n}_L{L}_r{r}_wd{wd}_epochs{epochs}"
                if os.path.exists(paramname+"testMSE.npy"):
                    testMSEs[r,n,L,wd,ln] = np.load(paramname+"testMSE.npy",allow_pickle=True).item()
                    trainMSEs[r,n,L,wd,ln] = np.load(paramname+"trainMSEs.npy",allow_pickle=True)
                    weightdecays[r,n,L,wd,ln] = np.load(paramname+"weightdecays.npy",allow_pickle=True)
                    learningrates[r,n,L,wd,ln] = np.load(paramname+"learningrates.npy",allow_pickle=True)
                    files_found_list.append((r,n,L,wd,ln))
                else:
                    print(f"{paramname+'testMSE.npy'} not found")

In [4]:
files_found_list

[(1, 64, 2, 0.001, 0),
 (1, 64, 2, 0.001, 0.25),
 (1, 64, 2, 0.001, 0.5),
 (1, 64, 2, 0.001, 1),
 (1, 64, 2, 0.0001, 0),
 (1, 64, 2, 0.0001, 0.25),
 (1, 64, 2, 0.0001, 0.5),
 (1, 64, 2, 0.0001, 1),
 (1, 64, 2, 1e-05, 0),
 (1, 64, 2, 1e-05, 0.25),
 (1, 64, 2, 1e-05, 0.5),
 (1, 64, 2, 1e-05, 1),
 (1, 64, 3, 0.001, 0),
 (1, 64, 3, 0.001, 0.25),
 (1, 64, 3, 0.001, 0.5),
 (1, 64, 3, 0.001, 1),
 (1, 64, 3, 0.0001, 0),
 (1, 64, 3, 0.0001, 0.25),
 (1, 64, 3, 0.0001, 0.5),
 (1, 64, 3, 0.0001, 1),
 (1, 64, 3, 1e-05, 0),
 (1, 64, 3, 1e-05, 0.25),
 (1, 64, 3, 1e-05, 0.5),
 (1, 64, 3, 1e-05, 1),
 (1, 64, 4, 0.001, 0),
 (1, 64, 4, 0.001, 0.25),
 (1, 64, 4, 0.001, 0.5),
 (1, 64, 4, 0.001, 1),
 (1, 64, 4, 0.0001, 0),
 (1, 64, 4, 0.0001, 0.25),
 (1, 64, 4, 0.0001, 0.5),
 (1, 64, 4, 0.0001, 1),
 (1, 64, 4, 1e-05, 0),
 (1, 64, 4, 1e-05, 0.25),
 (1, 64, 4, 1e-05, 0.5),
 (1, 64, 4, 1e-05, 1),
 (1, 64, 5, 0.001, 0),
 (1, 64, 5, 0.001, 0.25),
 (1, 64, 5, 0.001, 0.5),
 (1, 64, 5, 0.001, 1),
 (1, 64, 5, 0.0001

In [5]:
def Llayers(L,d=20,width=1000):
    #construct L-1 linear layers; bias term only on last linear layer
    if L < 2:
        raise ValueError("L must be at least 2")
    if L == 2:
        linear_layers = [nn.Linear(d,width,bias=True)]
    if L > 2:
        linear_layers = [nn.Linear(d,width,bias=False)]
        for l in range(L-3):
            linear_layers.append(nn.Linear(width,width,bias=False))
        linear_layers.append(nn.Linear(width,width,bias=True))

    relu = nn.ReLU()

    last_layer = nn.Linear(width,1)

    layers = linear_layers + [relu,last_layer]

    return nn.Sequential(*layers)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
files_found_list = []
models = {}
for r,n in rnvals:
    for L in Ls:
        for wd in wds:
            for ln in labelnoise:
                paramname = job_name+f"_labelnoise{ln}/N{n}_L{L}_r{r}_wd{wd}_epochs{epochs}"
                if os.path.exists(paramname+"model.pt"):
                    models[r,n,L,wd,ln] = Llayers(L,width=1000)
                    models[r,n,L,wd,ln].to(device)
                    if torch.cuda.is_available():
                        models[r,n,L,wd,ln].load_state_dict(torch.load(paramname+"model.pt"))
                    else:
                        models[r,n,L,wd,ln].load_state_dict(torch.load(paramname+"model.pt"),map_location=torch.device('cpu'))
                    models[r,n,L,wd,ln].eval()
                    files_found_list.append((r,n,L,wd,ln))
                else:
                    print(paramname+"model.pt","not found")

In [ ]:
files_found_list

## create pandas table

In [ ]:
res = {
  "r"                                    : [r                          for r,n,L,wd,ln in files_found_list],# + [r                         for wd in wds for r,n in relu_rnvals],
  "sigma"                                : [ln                         for r,n,L,wd,ln in files_found_list],# + [ln                        for wd in wds for r,n in relu_rnvals],
  "n"                                    : [n                          for r,n,L,wd,ln in files_found_list],# + [n                         for wd in wds for r,n in relu_rnvals],
  "L"                                    : [L                          for r,n,L,wd,ln in files_found_list],# + [4                         for wd in wds for r,n in relu_rnvals],
  "lambda"                               : [wd                         for r,n,L,wd,ln in files_found_list],# + [wd                        for wd in wds for r,n in relu_rnvals],
  "Learning Rate"                        : [learningrates[r,n,L,wd,ln] for r,n,L,wd,ln in files_found_list],# + [RELUlearningrates[r,n][4][wd] for wd in wds for r,n in relu_rnvals],
  "Train MSE"                            : [trainMSEs[r,n,L,wd,ln]     for r,n,L,wd,ln in files_found_list],# + [RELUtrainMSEs[r,n][4][wd]     for wd in wds for r,n in relu_rnvals],
  "Weight Decay"                         : [weightdecays[r,n,L,wd,ln]  for r,n,L,wd,ln in files_found_list],# + [RELUweightdecays[r,n][4][wd]  for wd in wds for r,n in relu_rnvals],
  "Model"                                : [models[r,n,L,wd,ln]        for r,n,L,wd,ln in files_found_list],# + [RELUmodels[r,n,4,wd]          for wd in wds for r,n in relu_rnvals],
  "Test MSE"                             : [testMSEs[r,n,L,wd,ln]      for r,n,L,wd,ln in files_found_list],# + [RELUtestMSEs[r,n][4][wd].item()      for wd in wds for r,n in relu_rnvals],
  "Activations"                          : ["linear and relu"          for r,n,L,wd,ln in files_found_list],# + ["relu only"         for wd in wds for r,n in relu_rnvals]
}
res = pd.DataFrame(res)
res["Final Train MSE"] = [r[-1] for r in res["Train MSE"]]
res["Final Weight Decay"] = [r[-1] for r in res["Weight Decay"]]
res

# Initial Training Time Checks


## Train MSE v Epoch


In [ ]:
# for ln in labelnoise:
#     for lamb in wds:
#         f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
#         plt.figure(figsize=(10,10))
#         for rownum,row in res.iterrows():
#             if row["L"] <= 4 and row["sigma"] == ln and row["lambda"] == lamb:
#                 whichrow = np.where(row['r'] == res.r.unique())[0][0]
#                 whichcol = np.where(row['n'] == res.n.unique())[0][0]
#                 ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
#                 ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
#                 ax[whichrow,whichcol].set_xlabel("Epoch")
#         ax[0,0].legend()
#         f.suptitle(rf"Train MSE v Epoch (label noise $N(0,{ln**2})$, weight decay $\lambda$ = {lamb})")
#         f.savefig(job_name+f"_labelnoise{ln}/trainmse234_lamb{lamb}.png",dpi=300)
#         plt.show()
#         f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
#         plt.figure(figsize=(10,10))
#         for rownum,row in res.iterrows():
#             if 4 < row["L"] <= 7 and row["sigma"] == ln and row["lambda"] == lamb:
#                 whichrow = np.where(row['r'] == res.r.unique())[0][0]
#                 whichcol = np.where(row['n'] == res.n.unique())[0][0]
#                 ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
#                 ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
#                 ax[whichrow,whichcol].set_xlabel("Epoch")
#         ax[0,0].legend()
#         f.suptitle(rf"Train MSE v Epoch (label noise $N(0,{ln**2})$, weight decay $\lambda$ = {lamb})")
#         f.savefig(job_name+f"_labelnoise{ln}/trainmse567_lamb{lamb}.png",dpi=300)
#         plt.show()
#         f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))
#         plt.figure(figsize=(10,10))
#         for rownum,row in res.iterrows():
#             if row["L"] > 7 and row["sigma"] == ln and row["lambda"] == lamb:
#                 whichrow = np.where(row['r'] == res.r.unique())[0][0]
#                 whichcol = np.where(row['n'] == res.n.unique())[0][0]
#                 ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
#                 ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
#                 ax[whichrow,whichcol].set_xlabel("Epoch")
#         ax[0,0].legend()
#         f.suptitle(rf"Train MSE v Epoch (label noise $N(0,{ln**2})$, weight decay $\lambda$ = {lamb})")
#         f.savefig(job_name+f"_labelnoise{ln}/trainmse8910_lamb{lamb}.png",dpi=300)
#         plt.show()

## Weight Decay v Epoch


In [ ]:
# for ln in labelnoise:
#     for lamb in wds:
#         f, ax = plt.subplots(len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
#         plt.figure(figsize=(10,10))
#         for rownum,row in res.iterrows():
#             if row["sigma"] == ln and row["lambda"] == lamb:
#                 whichrow = np.where(row['r'] == res.r.unique())[0][0]
#                 whichcol = np.where(row['n'] == res.n.unique())[0][0]
#                 ax[whichrow,whichcol].semilogy(row["Weight Decay"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.7)
#                 ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
#                 ax[whichrow,whichcol].set_xlabel("Epoch")
#         ax[0,0].legend()
#         f.suptitle(rf"Weight Decay v Epoch (label noise $N(0,{ln**2})$, weight decay $\lambda$ = {lamb})")
#         f.savefig(job_name+f"_labelnoise{ln}/weightdecay_lamb{lamb}.png",dpi=300)
#         plt.show()

## learning rates


In [ ]:
# for ln in labelnoise:
#     for lamb in wds:
#         f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
#         plt.figure(figsize=(10,10))
#         for rownum,row in res.iterrows():
#             if row["sigma"] == ln and row["lambda"] == lamb:
#                     whichrow = np.where(row['r'] == res.r.unique())[0][0]
#                     whichcol = np.where(row['n'] == res.n.unique())[0][0]
#                     ax[whichrow,whichcol].semilogy(row["Learning Rate"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.7)
#                     ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
#                     ax[whichrow,whichcol].set_xlabel("Epoch")
#         ax[0,0].legend()
#         f.suptitle(rf"Learning Rate v Epoch (label noise $N(0,{ln**2})$, weight decay $\lambda$ = {lamb})")
#         f.savefig(job_name+f"_labelnoise{ln}/LearningRate_lamb{lamb}.png",dpi=300)
#         plt.show()

# filter out bad training losses

In [ ]:
trainMSE_threshold = 1e-2

In [ ]:
res[res["Final Train MSE"] >= trainMSE_threshold + res["sigma"]] #TODO is this reasonable?

In [ ]:
res = res[res["Final Train MSE"] < trainMSE_threshold + res["sigma"]]
res

# generate data function


In [ ]:
def gen_data(datasetsize,r,seed,std,labelnoiseseed,trainsize=2**18,testsize=2**10,d=20,funcseed=42,verbose=False,ood=False):

    ##Generate data with a true central subspaces of varying dimensions
    #generate X values for training and test sets
    np.random.seed(seed) #set seed for data generation
    trainX = np.random.rand(d,trainsize).astype(np.float32)[:,:datasetsize] - 0.5 #distributed as U[-1/2, 1/2]
    testX = np.random.rand(d,testsize).astype(np.float32) - 0.5 #distributed as U[-1/2, 1/2]
    #out of distribution datagen
    if ood:
        trainX *= 2 #now distributed as U[-1, 1]
        testX *= 2 #now distributed as U[-1, 1]
    ##for each $r$ value create and store data-gen functions and $y$ evaluations
    #geneate params for functions
    k = d+1
    U = np.load(job_name+f"_labelnoise{std}/r{r}U.npy")
    Sigma = np.load(job_name+f"_labelnoise{std}/r{r}Sigma.npy")
    V = np.load(job_name+f"_labelnoise{std}/r{r}V.npy")
    A = np.load(job_name+f"_labelnoise{std}/r{r}A.npy")
    B = np.load(job_name+f"_labelnoise{std}/r{r}B.npy")
    #create functions
    np.random.seed(labelnoiseseed) #set seed for data generation
    def g(z): #active subspace function
        hidden_layer = (U*Sigma)@z
        hidden_layer = hidden_layer.T + B
        hidden_layer = np.maximum(0,hidden_layer).T
        return A@hidden_layer
    def f(x): #teacher network
        z = V.T@x    
        eps = std*np.random.randn(x.shape[1])    
        return g(z) + eps
    #generate data
    trainY = f(trainX).astype(np.float32)
    testY = f(testX).astype(np.float32)
    #move data to device
    if verbose:
        print("device: {}".format(device))
    trainX = torch.from_numpy(trainX).T.to(device)
    trainY = torch.from_numpy(trainY).to(device)
    testX = torch.from_numpy(testX).T.to(device)
    testY = torch.from_numpy(testY).to(device)
    if verbose:
        print("trainX shape = {} trainY shape = {}".format(
            trainX.shape,
            trainY.shape
        ))
    return trainX,trainY,testX,testY


# Validation MSE


## generate data


In [ ]:
validationY = {}

validationsize = 2048
for r in rs:
    for k,std in enumerate(labelnoise):
        labelnoiseseed = 686 + k
        datagenseed = 1107
        print("validation size =",validationsize,"r =",r,"label noise std =",std,"label noise seed =",labelnoiseseed)
        validationX,validationY[r,std] = gen_data(datasetsize=validationsize,r=r,seed=datagenseed,std=std,labelnoiseseed=labelnoiseseed)[:2]
validationX.min(),validationX.max()


## compute squared errors


In [ ]:
with torch.no_grad():
    validation = []
    normalized_validation = []
    squared_errors = []
    validation_sem = []
    validation_std = []
    for rownum, row in res.iterrows():
        std = row["sigma"]
        predY = row["Model"](validationX)
        squared_err = (predY[:,0] - validationY[row["r"],std])**2
        squared_err = squared_err.cpu().numpy()
        mse = nn.functional.mse_loss(predY[:,0],validationY[row["r"],std]).item()
        assert np.isclose(mse,np.mean(squared_err))
        validation.append(mse)
        sem_sqared_err = sem(squared_err)
        validation_sem.append(sem_sqared_err)
        std_sqared_err = np.std(squared_err)
        validation_std.append(std_sqared_err)
        if std > 0:
            normalized_validation.append(mse/(std**2))
        else:
            normalized_validation.append(np.nan)
        squared_errors.append(squared_err)
    res["Validation MSE"] = validation
    res["Validation MSE$/\sigma^2$"] = normalized_validation
    res["Validation Squared Errors"] = squared_errors
    res["Validation SEM"] = validation_sem
    res["Validation STD of Squared Errors"] = validation_std
res

# Generalization MSE

## generate data

In [ ]:
generalizationY = {}
generalizationsize = 2048
for r in rs:
    for k,std in enumerate(labelnoise):
        labelnoiseseed = 743 + k
        datagenseed = 555
        print("generalization size =",generalizationsize,"r =",r,"label noise std =",std,"label noise seed =",labelnoiseseed)
        generalizationX,generalizationY[r,std] = gen_data(datasetsize=generalizationsize,r=r,seed=datagenseed,std=std,labelnoiseseed=labelnoiseseed)[:2]

In [ ]:
generalizationX.min(),generalizationX.max()

## compute squared errors

In [ ]:
with torch.no_grad():
    generalization = []
    normalized_generalization = []
    squared_errors = []
    generalization_sem = []
    generalization_std = []
    for rownum, row in res.iterrows():
        std = row["sigma"]
        predY = row["Model"](generalizationX)
        squared_err = (predY[:,0] - generalizationY[row["r"],std])**2
        squared_err = squared_err.cpu().numpy()
        mse = nn.functional.mse_loss(predY[:,0],generalizationY[row["r"],std]).item()
        assert np.isclose(mse,np.mean(squared_err))
        generalization.append(mse)
        sem_sqared_err = sem(squared_err)
        generalization_sem.append(sem_sqared_err)
        std_sqared_err = np.std(squared_err)
        generalization_std.append(std_sqared_err)
        if std > 0:
            normalized_generalization.append(mse/(std**2))
        else:
            normalized_generalization.append(np.nan)
        squared_errors.append(squared_err)
    res["Generalization MSE"] = generalization
    res["Generalization MSE$/\sigma^2$"] = normalized_generalization
    res["Generalization Squared Errors"] = squared_errors
    res["Generalization SEM"] = generalization_sem
    res["Generalization STD of Squared Errors"] = generalization_std

# Out of Distribution MSE

## generate data

In [ ]:
oodY = {}
oodsize = 2048
for r in rs:
    for k,std in enumerate(labelnoise):
        labelnoiseseed = 235 + k
        datagenseed = 333
        print("ood size =",oodsize,"r =",r,"label noise std =",std,"label noise seed =",labelnoiseseed)
        oodX,oodY[r,std] = gen_data(datasetsize=oodsize,r=r,seed=datagenseed,std=std,labelnoiseseed=labelnoiseseed,ood=True)[:2]

In [ ]:
oodX.min(),oodX.max()

## compute squared errors

In [ ]:
with torch.no_grad():
    ood = []
    normalized_ood = []
    squared_errors = []
    ood_sem = []
    ood_std = []
    for rownum, row in res.iterrows():
        std = row["sigma"]
        predY = row["Model"](oodX)
        squared_err = (predY[:,0] - oodY[row["r"],std])**2
        squared_err = squared_err.cpu().numpy()
        mse = nn.functional.mse_loss(predY[:,0],oodY[row["r"],std]).item()
        assert np.isclose(mse,np.mean(squared_err))
        ood.append(mse)
        sem_sqared_err = sem(squared_err)
        ood_sem.append(sem_sqared_err)
        std_sqared_err = np.std(squared_err)
        ood_std.append(std_sqared_err)
        if std > 0:
            normalized_ood.append(mse/(std**2))
        else:
            normalized_ood.append(np.nan)
        squared_errors.append(squared_err)
    res["Out of Distribution MSE"] = ood
    res["Out of Distribution MSE$/\sigma^2$"] = normalized_ood
    res["Out of Distribution Squared Errors"] = squared_errors
    res["Out of Distribution SEM"] = ood_sem
    res["Out of Distribution STD of Squared Errors"] = ood_std

# Check that most or all ReLU hyperplanes intersect the support of the distributions of the tests

In [ ]:
for r in rs:
    print("r =",r)
    U = np.load(job_name+f"_labelnoise0/r{r}U.npy")
    Sigma = np.load(job_name+f"_labelnoise0/r{r}Sigma.npy")
    V = np.load(job_name+f"_labelnoise0/r{r}V.npy")
    A = np.load(job_name+f"_labelnoise0/r{r}A.npy")
    B = np.load(job_name+f"_labelnoise0/r{r}B.npy")
    W = (U*Sigma)@V.T
    rowwise1norms = np.linalg.norm(W,axis=1,ord=1)
    ratios = np.abs(B) / np.linalg.norm(W,axis=1,ord=1)
    rowwise2norms = np.linalg.norm(W,axis=1,ord=2)

    for datasetsize in res.n.unique():
        units = pd.DataFrame({"R2-cost contribution":np.abs(A)*rowwise2norms,"|b| / ||w||_1":ratios})
        # units["[-1/2,1/2]"] = ratios <= 1/2
        # units["[-1,1] but not [-1/2,1/2]"] = (1/2 < ratios) * (ratios <= 1)
        # units["not [-1,1]"] = ratios > 1
        trainX = gen_data(datasetsize=datasetsize,r=r,seed=1,std=0,labelnoiseseed=0)[0]
        units["# training active"] = ((W@trainX.cpu().numpy().T).T + B > 0).sum(axis=0)
        units["% training active"] = units["# training active"] / datasetsize
        units["# validation active"] = ((W@validationX.cpu().numpy().T).T + B > 0).sum(axis=0)
        units["% validation active"] = units["# validation active"] / datasetsize
        units["# generalization active"] = ((W@generalizationX.cpu().numpy().T).T + B > 0).sum(axis=0)
        units["% generalization active"] = units["# generalization active"] / datasetsize
        units["# ood active"] = ((W@oodX.cpu().numpy().T).T + B > 0).sum(axis=0)
        units["% ood active"] = units["# ood active"] / datasetsize

        print("\nTOTALS:\n~~~~~~~\n",units.sum())
        print("\nunit-wise table:\n~~~~~~~\n")
        display(units)
        plt.bar(units.index-0.2,units["# training active"]/datasetsize,label="train",width=0.4,tick_label=units["R2-cost contribution"].round(1))
        # plt.bar(units.index,units["# generalization active"]/datasetsize,label="gen",width=0.4)
        plt.bar(units.index+0.2,units["# ood active"]/datasetsize,label="ood",width=0.4)
        plt.ylim(0,1)
        plt.xticks(rotation=45)
        plt.title(f"How many samples is each unit active on? $r = {r}, n = {datasetsize}$")
        plt.xlabel("ReLU Unit, labeled by R2-cost contribution")
        plt.ylabel("Proportion of samples")
        plt.axhline(0.5,linestyle=":",color="k")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"samples_active_by_unit_r{r}_n{datasetsize}")
        plt.show()


# Active Subspace

## evaluate gradients and compute singular values and active subspaces

In [ ]:
grads = []
sv = []
active_subspace = []
subspace_dist = []

for rownum, row in res.iterrows():
    #compute ground truth active subspace
    funcseed = 42
    d = 20
    k = d+1
    ln = row['sigma']
    if int(ln) == ln:
        ln = int(ln)
    U = np.load(job_name+f"_labelnoise{ln}/r{row['r']}U.npy")
    Sigma = np.load(job_name+f"_labelnoise{ln}/r{row['r']}Sigma.npy")
    V = np.load(job_name+f"_labelnoise{ln}/r{row['r']}V.npy")
    W = np.load(job_name+f"_labelnoise{ln}/r{row['r']}W.npy")
    A = np.load(job_name+f"_labelnoise{ln}/r{row['r']}A.npy")
    B = np.load(job_name+f"_labelnoise{ln}/r{row['r']}B.npy")

    #evaluate gradients
    generalizationX.requires_grad = True
    predY = row["Model"](generalizationX)
    grad = torch.autograd.grad(predY, generalizationX,
                            grad_outputs=torch.ones_like(predY),
                            create_graph=True)[0].detach().cpu().numpy()
    grads.append(grad)
    #compute active subspace and singular values
    Uhat,Shat,VhatT = np.linalg.svd(grad)
    Vhat = VhatT.T[:,:row["r"]] #form the basis for the active subspace
    active_subspace.append(Vhat)
    sv.append(Shat)

    subspace_dist.append(np.linalg.norm(V@V.T - Vhat@Vhat.T,2))

res["Gradient Evaluations"] = grads
res["Gradient Singular Values"] = sv
res["Active Subspace"] = active_subspace
res["Active Subspace Distance"] = subspace_dist
res["Angle Error (Degrees)"] = np.degrees(np.arcsin(res["Active Subspace Distance"]))

res

# Tuning Hyperparameters

##  determine the lambda parameter that gets the best Validation MSE for each (r,n,L)

In [ ]:
validationmse_vs_lambda = res.pivot_table(values="Validation MSE",index = ("r","sigma","n","L","Activations"),columns=["lambda"])
validationmse_vs_lambda

In [ ]:
bestlambda = validationmse_vs_lambda.idxmin(axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(bestlambda)

In [ ]:
mask = [row["lambda"] == bestlambda[row["r"]][row["sigma"]][row["n"]][row["L"]][row["Activations"]] for rowindex,row in res.iterrows()]
res = res[mask]
res

##  determine the L parameter that gets the best validation MSE for each (r,n)

In [ ]:
validationmse_vs_L = res.pivot_table(values="Validation MSE",index = ("r","sigma","n","Activations"),columns=["L"])
validationmse_vs_L = validationmse_vs_L.iloc[:,1:]
bestL = validationmse_vs_L.idxmin(axis=1)
pd.concat((validationmse_vs_L,bestL),axis=1)

In [ ]:
mask = [row["L"] == bestL[row["r"]][row["sigma"]][row["n"]][row["Activations"]] for rowindex,row in res.iterrows()]
bestLres = res[mask]
bestLres

In [ ]:
bestLres.sort_values(by=['r','n',"sigma"])

## What are the chosen lambda and L for each model?

In [ ]:
bestres = pd.concat((res[res["L"] == 2],bestLres))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(bestres.loc[:,:"lambda"].pivot_table(index=["r","sigma","n","L"],values=["lambda"]))

# Plotting

## plots of all the singular values

In [ ]:
#all the singular values
for std in labelnoise:
    f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(10,4.8))
    for rownum,row in res.iterrows():
        if row['sigma'] == std:
            whichrow = np.where(row['r'] == res.r.unique())[0][0]
            whichcol = np.where(row['n'] == res.n.unique())[0][0]
            print(whichrow,whichcol)
            print(rf"{row['r']},{row['n']},{row['L']}",row["Gradient Singular Values"]/np.sqrt(2048),whichrow,whichcol)
            ax[whichrow,whichcol].semilogy(row["Gradient Singular Values"]/np.sqrt(2048),label=rf"$L={row['L']}$",linewidth=1,alpha=0.7,marker=".")
            ax[whichrow,whichcol].set_xticks(list(range(3,20,4)),list(range(4,21,4)))
            ax[whichrow,whichcol].set_ylim(10**-9,10**3)
            ax[0,whichcol].set_title(rf"$n={row['n']}$")
            ax[-1,whichcol].set_xlabel(rf"Index, $k$")
    plt.subplot(2,len(res.n.unique()),1)
    leg = plt.legend()
    leg = plt.legend(bbox_to_anchor=(-1, 1))
    leg.get_frame().set_edgecolor('b')
    leg.get_frame().set_linewidth(0.0)
    plt.subplot(2,len(res.n.unique()),1)
    plt.ylabel(r"$r=1$"+"\n"+r"$\sigma_k(\hat f;\rho)$")
    plt.yticks([10**p for p in range(-12,3,2)])
    plt.subplot(2,len(res.n.unique()),len(res.n.unique())+1)
    plt.ylabel(r"$r=2$"+"\n"+r"$\sigma_k(\hat f;\rho)$")
    plt.yticks([10**p for p in range(-12,3,2)])
    plt.suptitle(rf"Singular Values of Trained Networks, $\sigma =$ {std}")
    plt.tight_layout()
    plt.savefig(job_name+f"_labelnoise{std}/sv.png",dpi=300)#,bbox_extra_artists=(leg,), bbox_inches='tight')
    plt.show()

## Plots of L vs Validation error

In [ ]:
for std in res["sigma"].unique():
    f, ax = plt.subplots(ncols=len(res.r.unique()),nrows=1, sharex=True, sharey=False, figsize=(10,4.8))
    for rnum,r in enumerate(res.r.unique()):
        for n in res.n.unique():
                res_rnstd = res[(res.r == r) * (res.n == n) * (res["sigma"] == std)]
                ax[rnum].scatter(res_rnstd.L,res_rnstd[["Validation MSE"]])
                ax[rnum].semilogy(res_rnstd.L,res_rnstd[["Validation MSE"]],label=rf"$n={n}$")
                for _,model in res_rnstd.iterrows():
                    text = rf'$\lambda = {model["lambda"]:.0e}$' + f'\nfit {model["Final Train MSE"]:.1e}\nwd{model["Final Weight Decay"]:.1e}'
                    ax[rnum].annotate(text,[model.L,model[["Validation MSE"]]],fontsize=1)
        ax[rnum].set_xlabel("$L$ number of layers")
        ax[rnum].set_title(rf"$r={r}$")
        if std > 0:
            ax[rnum].axhline(y=std**2, color='k', linestyle=':',label="$\sigma^2$")
        ax[0].set_ylabel("Validation MSE")
        f.suptitle(rf"Validation MSE for best $\lambda$ values, $\sigma$ = {std}")
    ax[0].legend()
    f.tight_layout()
    if int(std) == std:
        std = int(std)
    f.savefig(job_name+f"_labelnoise{std}/ValidationMSE.png",dpi=300)
    plt.show()

## Performance metrics with/without linear layers

In [ ]:
colors = {  
    0   :"C0",
    0.25:"C1",
    0.5 :"C2",
    1   :"C3"
}
handles = [
    Line2D([0], [0], color=color, ls='-', label=rf"$\sigma =${sigma}") for sigma,color in colors.items()
]
labels = {
    "without linear layers":"-",
    "with linear layers"   :"--",
}
markers = {
    "without linear layers":".",
    "with linear layers"   :"x",
}
handles += [
    Line2D([0], [0], color='k', ls=ls, label=label, marker = markers[label], markersize=4) for label,ls in labels.items()
]

In [ ]:
ranktol = 1e-3
handles += [Line2D([0], [0], color='k', ls=':', label="rank tolerance cutoff")]
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(15,5))
for row,r in enumerate(res.r.unique()):
    for col,n in enumerate(res.n.unique()):
        for sigma in labelnoise:
            for label,ls in labels.items():
                if label == "with linear layers":
                    curr = bestLres[(bestLres.r == r) * (bestLres.n == n) * (bestLres["sigma"] == sigma)]
                elif label == "without linear layers":
                    curr = res[(res.L == 2) * (res.n == n) * (res.r == r) * (res["sigma"]==sigma)]
                print(r,n,label,sigma,curr["Gradient Singular Values"].values[0]/np.sqrt(2048))
                marker = markers[label]
                ax[row,col].semilogy(curr["Gradient Singular Values"].values[0]/np.sqrt(2048),linestyle=ls,linewidth=1,alpha=0.7,marker=marker,markersize=4,color=colors[sigma])
        ax[row,col].axhline(y=ranktol, color='k', linestyle=':',alpha=1, label = rf"rank tolerance cutoff, $\varepsilon = {ranktol}$")
        ax[row,col].set_xticks(list(range(3,20,4)),list(range(4,21,4)))
        ax[-1,col].set_xlabel(rf"Index, $k$")
        ax[row,0].set_ylabel(rf"$r={r}$"+"\n"+r"$\sigma_k(\hat f;\rho)$")
        ax[0,0].set_yticks([10**p for p in range(-9,3,2)])
        ax[row,col].set_ylim(10**-9,10**3)
        ax[0,col].set_title(rf"$n={n}$")
        ax[0,-1].legend(handles=handles, loc='best', bbox_to_anchor=(1.05,1))
plt.suptitle(rf"Singular Values of Trained Networks after $\lambda, L$ tuning")
plt.tight_layout()
plt.savefig(job_name+f"_labelnoise_sv.png",dpi=600)#,bbox_extra_artists=(leg,), bbox_inches='tight')
plt.show()
handles.pop()

In [ ]:
#active subspace error plot
f, ax = plt.subplots(ncols=len(res.r.unique()),nrows=2, sharex=True, sharey=False, figsize=(10,9))
for row,metric in enumerate([rf"Effective Rank, $\varepsilon = {ranktol}$","Angle Error (Degrees)"]):
    for col,r in enumerate(res.r.unique()):
        for sigma in [0,0.25,0.5,1]:
            ax[row,0].set_ylabel(f"{metric}")
            for label,ls in labels.items():
                if label == "with linear layers":
                    curr = bestLres[(bestLres.r == r) * (bestLres["sigma"] == sigma)]
                elif label == "without linear layers":
                    curr = res[(res.L == 2) * (res.r == r) * (res["sigma"]==sigma)]
                if metric == "Angle Error (Degrees)":
                    points = curr[[metric]].values[:,0]
                elif metric == rf"Effective Rank, $\varepsilon = {ranktol}$":
                    points = (np.array(curr["Gradient Singular Values"].tolist())/np.sqrt(2048) > ranktol).sum(axis=1)
                    ax[row,col].set_yticks(np.arange(0,21,2))
                    ax[row,col].set_ylim(0,20.5)
                marker = markers[label]
                ax[row,col].plot(curr.n,points,
                                    linestyle=ls,
                                    color=colors[sigma],
                                    marker=marker,
                                    markersize=4,
                                    alpha=0.8)
                for (_,model),y in zip(curr.iterrows(),points):
                    text = rf'$\lambda = {model["lambda"]:.0e}$' + f'\nL = {model["L"]}\nfit {model["Final Train MSE"]:.1e}\nwd{model["Final Weight Decay"]:.1e}\nVal{model["Validation MSE"]:.1e}'
                    ax[row,col].annotate(text,[model["n"],y],fontsize=1)
        #plot set up
        ax[0,col].set_title(rf"$r={r}$") 
        ax[0,1].legend(handles=handles, loc='best', bbox_to_anchor=(1.05,1))
        ax[row,col].set_xscale("log",base=2)
        ax[1,col].set_xlabel("$n$ number of samples")

plt.suptitle(f"Active Subspaces after $\lambda, L$ tuning")
plt.tight_layout()
plt.savefig(job_name+f"Active Subspaces after tuning lambda and L.png",dpi=600)
plt.show()

In [ ]:
#generalization and OOD errors
handles += [
        Line2D([0], [0], color='k', ls=':', label='$\sigma^2$, variance of label noise'),
]

for metric in ['Generalization MSE','Out of Distribution MSE']:
    standard_errors = metric[:-3] + 'SEM'
    f, ax = plt.subplots(ncols=len(res.r.unique()),nrows=2, sharex=True, sharey="row", figsize=(10,9))
    #just the data without label noise in the first row
    for col,r in enumerate(res.r.unique()):
        for row,sigmas in enumerate([[0],[0.25,0.5,1]]):
            for sigma in sigmas:
                for label,ls in labels.items():
                    if label == "with linear layers":
                        curr = bestLres[(bestLres.r == r) * (bestLres["sigma"] == sigma)]
                    elif label == "without linear layers":
                        curr = res[(res.L == 2) * (res.r == r) * (res["sigma"]==sigma)]
                    points = curr[[metric]].values[:,0]
                    errorbars = curr[[standard_errors]].values[:,0]
                    marker = markers[label]
                    ax[row,col].plot(curr.n,points,
                                        linestyle=ls,
                                        marker=marker,
                                        markersize=4,
                                        color=colors[sigma],
                                        alpha=0.8)
                    #horizontal dashed line for minimal possible MSE (ie sigma^2) in plots with label noise
                    ax[1,col].axhline(y=sigma**2, color=colors[sigma], linestyle=':',alpha=0.3)
                    for (_,model),y in zip(curr.iterrows(),points):
                        text = rf'$\lambda = {model["lambda"]:.0e}$' + f'\nL = {model["L"]}\nfit {model["Final Train MSE"]:.1e}\nwd{model["Final Weight Decay"]:.1e}\nVal{model["Validation MSE"]:.1e}'
                        ax[row,col].annotate(text,[model["n"],y],fontsize=1)
            #plot set up
            ax[row,col].set_xscale("log",base=2)
            ax[row,col].set_yscale("log",base=10)
            ax[row,0].set_ylabel(f"{metric}")
            ax[0,col].set_title(rf"$r={r}$") 
            ax[1,col].set_xlabel("$n$ number of samples")
            ax[0,1].legend(handles=handles, loc='best', bbox_to_anchor=(1.05,1))
    plt.suptitle(f"{metric}" + r" after $\lambda, L$ tuning")
    plt.tight_layout()
    plt.savefig(job_name+f"{metric}.png",dpi=600)
    plt.show()

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     pd.options.display.float_format = '{:.1e}'.format
#     display(bestres.loc[:,:"Generalization MSE"].pivot_table(index=["r","sigma","n","L"],values=["Generalization MSE"])-0.25)

## Visualizing Functions in Low Dimensions

In [ ]:
def get_ground_truth_func(r,std=0):
    #geneate params for functions
    k = d+1
    U = np.load(job_name+f"_labelnoise{std}/r{r}U.npy")
    Sigma = np.load(job_name+f"_labelnoise{std}/r{r}Sigma.npy")
    V = np.load(job_name+f"_labelnoise{std}/r{r}V.npy")
    A = np.load(job_name+f"_labelnoise{std}/r{r}A.npy")
    B = np.load(job_name+f"_labelnoise{std}/r{r}B.npy")
    #create functions
    def g(z): #active subspace function
        hidden_layer = (U*Sigma)@z
        hidden_layer = hidden_layer.T + B
        hidden_layer = np.maximum(0,hidden_layer).T
        return A@hidden_layer
    def f(x): #teacher network
        z = V.T@x    
        return g(z)
    return f

In [ ]:
r = 1
f = get_ground_truth_func(r=r) #ground truth function
v = np.load(job_name+f"_labelnoise{std}/r{r}V.npy")[:,0]

plot of projetion onto true 1d subspace of both models and ground truth

In [ ]:
def onedimproj(f,v,z,istorch=True):
    """
    Create a projection of the function onto 1D inputs. More specifically, return f(vz')
    """
    #create points vz at which to evaluate f
    vz = np.outer(v, z)
    if istorch:
        vz = np.float32(vz)
        vz = torch.from_numpy(vz).to(device).T
        #function evals
        with torch.no_grad():
            fvz = f(vz)
            fvz = fvz.cpu().numpy()
    else:
        #function evals
        fvz = f(vz)
    return fvz

In [ ]:
fig = plt.figure(figsize = (10,6))
fig.suptitle(f"Ground Truth vs. Models, $r = {r}$, Projected onto True Subspace")
ax = fig.add_subplot(1,1,1)
z = np.linspace(-0.1,0.1,500)
#plotting ground truth
ax.plot(z,g(z.reshape(1,-1)),label="Ground Truth")
#plotting models
for rownum,row in bestLres[(bestLres.r ==1)*(bestLres.sigma ==0)*(bestLres.n >= 1024)].iterrows():
    ax.plot(z,onedimproj(row.Model,v,z),label=rf"$\sigma={row.sigma}, n={row.n}$",alpha=0.5)
plt.legend()
plt.savefig(f"ooderrors_onedimvizr{r}")
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
fig.suptitle(f"Ground Truth - Models, $r = {r}$, Projected onto True Subspace")
ax = fig.add_subplot(1,1,1)
z = np.linspace(-0.4,0.4,500)
#plotting models
for rownum,row in bestLres[(bestLres.r ==1)*(bestLres.sigma ==0)*(bestLres.n >= 1024)].iterrows():
    ax.plot(z,onedimproj(row.Model,v,z)[:,0]-g(z.reshape(1,-1)),label=rf"$\sigma={row.sigma}, n={row.n}$",alpha=0.5)
plt.legend()
plt.savefig(f"ooderrors_onedimvizr{r}")
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
fig.suptitle(f"Ground Truth - Models, $r = {r}$, Projected onto True Subspace")
ax = fig.add_subplot(1,1,1)
z = np.linspace(-3,3,500)
#plotting models
for rownum,row in bestLres[(bestLres.r ==1)*(bestLres.sigma ==0)*(bestLres.n >= 1024)].iterrows():
    ax.plot(z,onedimproj(row.Model,v,z)[:,0]-g(z.reshape(1,-1)),label=rf"$\sigma={row.sigma}, n={row.n}$",alpha=0.5)
plt.legend()
plt.savefig(f"ooderrors_onedimvizr{r}")
plt.show()

plot of projetion onto true 1d subspace of both models and ground truth

In [ ]:
def projviz(ax,f,d=20,domain=None,projseed=0,istorch=True,verbose=False,kind="surface",**kwargs):
    """
    Plot a projection of the function onto 2D inputs. More specifically, plot visualization of a 
    function f with d dimensional input and 1 dimension output by plotting f(Az) where A is a 
    random d x 2 matrix with orthonormal columns and z is a 2-dimensional meshgrid.
    """
    #create a random projection matrix A
    A = ortho_group(d,seed=projseed).rvs()[:,:2]
    Asupnorm = np.linalg.norm(A,ord=np.inf)
    if domain is None:
        domain = np.linspace(-1,1,500)
        # domain = np.linspace(-2/Asupnorm,2/Asupnorm,500)
    if verbose: print("random projection d x 2:\n",A)
    #create 2D meshgrid of domain
    Z1,Z2 = np.meshgrid(domain,domain)
    numpnts = len(domain)
    #create points Az at which to evaluate f
    AZ = np.outer(A[:,0], Z1) + np.outer(A[:,1], Z2)
    if istorch:
        AZ = np.float32(AZ)
        AZ = torch.from_numpy(AZ).to(device).T
        #function evals
        with torch.no_grad():
            fAZ = f(AZ)
            fAZ = fAZ.cpu().numpy()
    else:
        #function evals
        fAZ = f(AZ)
    fAZ  = fAZ.reshape(numpnts,numpnts)
    #plotting -- using a surface plot, a wireframe plot, or a contour plot
    if kind=="surface":
        ax.plot_surface(Z1,Z2,fAZ,**kwargs)
    elif kind=="wire":
        ax.plot_wireframe(Z1,Z2,fAZ,**kwargs)
    elif kind=="contour":
        ax.contour(Z1,Z2,fAZ,**kwargs)
    elif kind=="contourf":
        ax.contourf(Z1,Z2,fAZ,**kwargs)

In [ ]:
for projseed in range(1,4):
    fig = plt.figure(figsize = (20,10))
    fig.suptitle(f"Ground Truth vs. Models, $r = {r}$, Projection {projseed}")

    # kind = "surface"
    # #plotting ground truth
    # ax = fig.add_subplot(1,3,1,projection='3d')
    # projviz(ax,f,istorch=False,kind=kind,projseed=projseed)
    # ax.set_title(f"Ground Truth")
    # #plotting models
    # subplot=2
    # for rownum,row in bestLres[(bestLres.r ==1)*(bestLres.sigma ==0)*(bestLres.n >= 1024)].iterrows():
    #     ax = fig.add_subplot(1,3,subplot,projection='3d')
    #     subplot += 1
    #     projviz(ax,row.Model,kind=kind,projseed=projseed)
    #     ax.set_title(rf"$\sigma={row.sigma}, n={row.n}$")
    # plt.show()

    kind = "contour"
    fig = plt.figure(figsize = (20,6))
    fig.suptitle(f"Ground Truth vs. Models, $r = {r}$, Projection {projseed}")
    #plotting ground truth
    ax = fig.add_subplot(1,3,1)
    projviz(ax,f,istorch=False,kind=kind,projseed=projseed,levels=100)
    ax.set_title(f"Ground Truth")
    #plotting models
    subplot=2
    for rownum,row in bestLres[(bestLres.r ==1)*(bestLres.sigma ==0)*(bestLres.n >= 1024)].iterrows():
        ax = fig.add_subplot(1,3,subplot)
        subplot += 1
        projviz(ax,row.Model,kind=kind,projseed=projseed,levels=100)
        ax.set_title(rf"$\sigma={row.sigma}, n={row.n}$")
    plt.savefig(f"ooderrors_projvizr{r}projection{projseed}")
    plt.show()

## Training v Epoch after Tuning $(\lambda, L)$

In [ ]:
sigmas = np.array([0,0.25,0.5,1])
f, ax = plt.subplots(nrows=2, ncols=4, sharex=True, sharey=True, figsize=(20,10))
plt.figure(figsize=(10,10))
for rownum,row in bestLres.iterrows():
    sigma = row['sigma']
    r = row['r']
    whichcol = np.where(sigma == sigmas)[0][0]
    whichrow = 0 if r == 1 else 1
    ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$n = {row['n']}$",linewidth=1,alpha=0.7)
    ax[whichrow,whichcol].set_title(rf"$r = {row['r']},\sigma = {sigma}$")
    ax[whichrow,whichrow].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle(rf"Train MSE v Epoch (Tuned $\lambda,L$)")
f.savefig(job_name+f"trainmse.png",dpi=300)
plt.show()

In [ ]:
sigmas = np.array([0,0.25,0.5,1])
f, ax = plt.subplots(nrows=2, ncols=4, sharex=True, sharey=True, figsize=(20,10))
plt.figure(figsize=(10,10))
for rownum,row in bestLres.iterrows():
    sigma = row['sigma']
    r = row['r']
    whichcol = np.where(sigma == sigmas)[0][0]
    whichrow = 0 if r == 1 else 1
    ax[whichrow,whichcol].semilogy(row["Weight Decay"],label=rf"$n = {row['n']}$",linewidth=1,alpha=0.7)
    ax[whichrow,whichcol].set_title(rf"$r = {row['r']},\sigma = {sigma}$")
    ax[whichrow,whichrow].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle(rf"Weight Decay v Epoch (Tuned $\lambda,L$)")
f.savefig(job_name+f"weightdecay.png",dpi=300)
plt.show()